In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
import argparse
import sys
import hail as hl

In [ ]:
#arguments
#def check_arg(args=None):
#    parser = argparse.ArgumentParser(description="import hail table from phenotype accession number")
#    parser.add_argument('-p', '--phecode', help='path to input file', required='True')
#    return parser.parse_args(args)

#args = check_arg(sys.argv[1:])
phenotype_id = 714.1
#phenotype_id = args.phecode

In [ ]:
#check if phecode file exists
path = f"gs://fc-aou-datasets-controlled/AllxAll/v1/ht/ACAF/META/phenotype_{phenotype_id}_ACAF_results.ht"

result = os.system(f"gsutil -u $GOOGLE_PROJECT ls {path} > /dev/null 2>&1")

if result == 0:
    print(f"Phenotype {phenotype_id} is in the All of Us database")
else:
    print(f"Phenotype {phenotype_id} is not in the All of Us database; enter valid phenotype ID")
    sys.exit(1)

In [ ]:
#initialize hail
hl.init()

#define bucket to save to
bucket = os.getenv('WORKSPACE_BUCKET')
bucket # gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/

#not necessary, shows files in bucket searched
os.system(f"gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/AllxAll/v1/ht/ACAF/META/phenotype_{phenotype_id}_ACAF_results.ht")

#find hail table and save to variable
ht = hl.read_table(f"gs://fc-aou-datasets-controlled/AllxAll/v1/ht/ACAF/META/phenotype_{phenotype_id}_ACAF_results.ht")

In [ ]:
#show first few lines of hail table
ht.show(20)

#summary of table
print(ht.describe())

#table dimentions
rows, cols = ht.count(), len(ht.row)
print(f"Table dimensions: {rows} rows x {cols} columns")

In [ ]:
#set thresholds
pval = 0.05
max_snps = 1000000
min_pval = 5e-8

#intial thinning of SNPs
significant_snps = ht.filter(ht.Pvalue < pval)
num_snps = significant_snps.count()

#continue to thin until less than max_snps threshold
while num_snps > max_snps and pval > min_pval:
    #make threshold 5x more stringent
    pval /= 5
    if pval < min_pval:
        pval = min_pval
    #thin again
    significant_snps = ht.filter(ht.Pvalue < pval)
    num_snps = significant_snps.count()
    if pval == min_pval:
        break

#print number of SNPs and pvalue in filtered table
print(f"Number of SNPs: {num_snps} at pvalue < {pval}")


#sort SNPs
sig_snps_sorted = significant_snps.order_by(significant_snps.Pvalue)

#print top SNPs
sig_snps_sorted.show(10)

#save filtered file
filtered_gt_path = f'{bucket}/data/filtered_{phenotype_id}.tsv'
sig_snps_sorted.export(filtered_gt_path)

In [ ]:
#to view contents of bucket
os.system(f"gsutil -u $GOOGLE_PROJECT ls gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data")

#to remove file from bucket
#os.system("gsutil rm gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/file")

#to view file in bucket (terminal)
#gsutil cat gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/401.tsv | head -n 10